In [1]:
import pandas as pd
from torch import nn
import torch
import numpy as np
import random
import  matplotlib.pyplot as plt
from torch.autograd import Function
from torch.autograd import Variable
from datetime import datetime
import math
import csv

In [2]:
all_data = pd.read_csv("../../SongApp.csv",encoding ='utf_8',delimiter=',')

In [20]:
test_data = pd.read_csv("../../SongTst.csv",encoding ='utf_8',delimiter=',')

In [21]:
test_data.head()

,Unnamed: 0,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
0,463715,52.67814,-2.88914,43.95268,-1.39209,-14.93379,-15.86877,1.19379,0.31401,-4.44235,...,-5.74356,-42.57910,-2.91103,48.72805,-3.08183,-9.38888,-7.27179,-4.00966,-68.96211,-5.21525
1,463716,45.74235,12.02291,11.03009,-11.60763,11.80054,-11.12389,-5.39058,-1.11981,-7.74086,...,-4.70606,-24.22599,-35.22686,27.77729,15.38934,58.20036,-61.12698,-10.92522,26.75348,-5.78743
2,463717,52.55883,2.87222,27.38848,-5.76235,-15.35766,-15.01592,-5.86893,-0.31447,-5.06922,...,-8.35215,-16.86791,-10.58277,40.10173,-0.54005,-11.54746,-45.35860,-4.55694,-43.17368,-3.33725
3,463718,51.34809,9.02702,25.33757,-6.62537,0.03367,-12.69565,-3.13400,2.98649,-6.71750,...,-6.87366,-20.03371,-66.38940,50.56569,0.27747,67.05657,-55.58846,-7.50859,28.23511,-0.72045
4,463719,45.84640,2.83376,-6.00506,-15.16150,-10.72385,-15.15233,5.00724,1.69039,-0.95527,...,-27.97255,-76.79480,55.54104,88.86441,-8.43241,62.00507,123.56146,7.87100,-38.61680,26.41166


In [3]:
all_data = all_data.drop(['Unnamed: 0'],axis = 1, inplace = False)

In [6]:
mat = all_data.as_matrix()

In [7]:
new_data_l = []
for i in range(100000):
    indice = random.randrange(0,len(mat))
    ligne = mat[indice]
    new_data_l.append(ligne)
new_data = pd.DataFrame(new_data_l)

In [24]:
def normalise_labels(labels):
    new_label = []
    mini = min(labels)
    maxi = max(labels)
    for i in range(len(labels)):
        new_label.append((labels[i]-mini)/(maxi-mini))
    return new_label,maxi,mini

def normalise_data(data):
    maxi = np.array(data).max()
    mini = np.array(data).min()
    new_data = []
    for i in range(len(data)):
        ligne = []
        for j in range(len(data[i])):
            ligne.append((data[i][j]-mini)/(maxi-mini))
        new_data.append(ligne)
    return new_data

def taux_erreur(modele ,data,label,loss):
    erreur = 0
    for i in range(len(data)):
        x = torch.FloatTensor(data[i])
        x = Variable(x.view(-1,taille_vect),requires_grad=True)
        y = torch.FloatTensor(1)
        y.zero_()
        y[0] = (label[i])
        y = Variable(y,requires_grad=False)
        y_pred = modele(x)
        err  = loss(y_pred , y)
        erreur+= err.data[0]
    return erreur/(len(data)*1.0)

In [25]:
labels,maxi,mini = normalise_labels(new_data[0].as_matrix())
data = normalise_data(new_data.drop([0],axis = 1, inplace = False).as_matrix())
taille_vect = len(data[0])

In [26]:
class Linear_Model(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.modele = torch.nn.Linear(X,Y)    
        
    def forward(self , x):
        return (torch.tanh(self.modele(x)))

In [27]:
class Out_Model(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.modele = torch.nn.Linear(X,Y)
        
    def forward(self , x):
        return (torch.sigmoid(self.modele(x)))

In [31]:
def train(modele , loss,data,nb_iter,step,label):
    tab_err_train = []
    i = 0
    data_train = []
    label_train = []
    while i < nb_iter :
        indice = random.randint(0,len(data)-1)
        x = torch.FloatTensor(data[indice])
        x = Variable(x.view(-1,taille_vect))
        y = torch.FloatTensor(final_out)
        y[0] = label[indice]
        y = Variable(y)
        y_pred = modele(x)
        erreur  = loss(y_pred , y)
        modele.zero_grad()
        erreur.backward()
        for param in modele.parameters():
            param.data -= step * param.grad.data
        i+=1
        if(i % 2 ==0):
            err = taux_erreur(modele ,data,label,loss)
            print ('    Erreur en train : ',err)
            print(' iter : ',i)
    return tab_err_train

In [29]:
final_out = 1
output = 1500#nb neuronne couche caché
models = {}
loss = torch.nn.MSELoss()
modele = torch.nn.Sequential(Linear_Model(taille_vect,output),Linear_Model(output,output),Out_Model(output,final_out))

In [ ]:
step = 0.01
nb_iter =100
tab_err_train = train(modele,loss,data,nb_iter,step,labels)

    Erreur en train :  0.0942472671997116
 iter :  2
    Erreur en train :  0.08169375891850747
 iter :  4
    Erreur en train :  0.07408162646634962
 iter :  6
    Erreur en train :  0.06860907850604253
 iter :  8
    Erreur en train :  0.06105726836371302
 iter :  10
    Erreur en train :  0.05458048406869307
 iter :  12
    Erreur en train :  0.04802147540257266
 iter :  14
    Erreur en train :  0.04676839221942532
 iter :  16
    Erreur en train :  0.04190118255902229
 iter :  18
    Erreur en train :  0.04195687640671769
 iter :  20
    Erreur en train :  0.03823261424175677
 iter :  22
    Erreur en train :  0.0350900757313191
 iter :  24
    Erreur en train :  0.031727628110015255
 iter :  26
    Erreur en train :  0.029121084906712367
 iter :  28
    Erreur en train :  0.026534175116767363
 iter :  30
    Erreur en train :  0.02474351719382753
 iter :  32
    Erreur en train :  0.024676266145280606
 iter :  34
    Erreur en train :  0.02423297156937698
 iter :  36
    Erreur e

In [57]:
def inference(data,maxi,mini,label):
    x = torch.FloatTensor(data)
    x = Variable(x.view(-1,taille_vect))
    y_pred = modele(x)
    y_denormalise = round((y_pred*(maxi-mini)+mini).data[0][0])
    label_denormalise = (label*(maxi-mini)+mini)
    print ("Valeure predite : ",(y_denormalise)," Valeure attendue : ",label_denormalise)

In [65]:
for i in range(50):
    inference(data[i],maxi,mini,labels[i])